# THE KYOTO PROTOCOL AS A PRIMARY DRIVER OF GREENHOUSE GAS EMISSIONS EMBODIED IN INTERNATIONAL TRADE FLOWS
## Appendix B

_______________________________________

## Note:

This code requires the necessary environmental variables and data source to exist in order to run succesfully.

For questions, please contact the author:
david.baumann@students.unibe.ch or
david.baumann@outlook.com


_______________________________________

### Load required packages

In [ ]:
#pip install numpy              --upgrade
#pip install pandas             --upgrade
#pip install warnings           --upgrade
#pip install time               --upgrade
#pip install datetime           --upgrade
#pip install sys                --upgrade
#pip install pymrio             --upgrade
#pip install gme                --upgrade
#pip install plotly             --upgrade
#pip install kaleido            --upgrade

In [ ]:
import pandas as pd
import numpy as np

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import time
import datetime
import sys

import pymrio
import gme

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

_______________________________________
### Load & process EORA dataset

In [ ]:
print('Preparing import...', end="")
time_start_all = time.time()

eora_storage = '/Users/david/OneDrive/Universität/Masterarbeit/Data/mrio/eora26/v199.82';
data_gme = pd.DataFrame(columns=['iso3_o', 'iso3_d', 'ghg_co2eq', 'ghg_co2eq_fpa', 'year'])

graph_countries_data = pd.read_excel('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/incl_countries.xlsx')
graph_countries = graph_countries_data['country']
graph_ghg_pba = pd.DataFrame(graph_countries)
graph_ghg_fpa = pd.DataFrame(graph_countries)
graph_ghg_cba = pd.DataFrame(graph_countries)
graph_ghg_im = pd.DataFrame(graph_countries)
graph_ghg_ex = pd.DataFrame(graph_countries)
print(' starting...')

for year_i in range(1990, 2016):
    
    time_start_loop = time.time()
    
    ## Initialize MRIO
    print('Year ' + str(year_i) + ': Loading... mrio', end="")
    temp_mrio = pymrio.parse_eora26(year=year_i, path=eora_storage)
    temp_mrio.ghg_diag = temp_mrio.Q.diag_stressor(('GHG emissions from PRIMAPHIST','PRIMAP|KYOTOGHGAR4|TOTALexcludingLULUCF|GgCO2eq'))
    temp_mrio.calc_all()
    
    ## Calculate Emissions Flows and Accounts
    print(' / flows', end="")
    temp_fpa = pd.DataFrame(temp_mrio.ghg_diag.M.sort_index().dot(temp_mrio.Y.groupby(level='region', axis=1).sum().sort_index().T.stack()))
    temp_cba = pd.DataFrame(temp_mrio.ghg_diag.M.sort_index().dot(temp_mrio.Y.groupby(level='region', axis=1).sum().sort_index()))
    
    ## GME Dataframe
    print(' / gme', end="")
    temp_gme = pd.DataFrame(temp_cba.groupby(level='region', axis=0).sum().stack(0), columns = ['ghg_co2eq'])
    temp_gme = temp_gme.reset_index(level=1).rename(columns={'region': 'iso3_d'})
    temp_gme = temp_gme.reset_index(level=0).rename(columns={'region': 'iso3_o'})
    temp_gme['year'] = year_i
    temp_gme_fpa = pd.DataFrame(temp_fpa.groupby(level='region', axis=0).sum().stack(0), columns = ['ghg_co2eq_fpa'])
    temp_gme_fpa = temp_gme_fpa.reset_index(level=1).rename(columns={'region': 'iso3_d'})
    temp_gme_fpa = temp_gme_fpa.reset_index(level=0).rename(columns={'region': 'iso3_o'})
    temp_gme_fpa['year'] = year_i
    temp_gme = pd.merge(temp_gme, temp_gme_fpa, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    data_gme = data_gme.append(temp_gme, ignore_index=True)
    
    ## Country PBA
    print(' / pba', end="")
    temp_pba = pd.DataFrame(temp_mrio.ghg_diag.D_cba.sort_index().groupby(level='region', axis=0).sum().sum(axis=1).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'PBA MtCO2eq'}))
    temp_pba['year'] = year_i
    temp_pba = pd.merge(graph_countries_data, temp_pba, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_pba = pd.concat([graph_ghg_pba, temp_pba], ignore_index=True, axis=0, join='outer')
    graph_ghg_pba = graph_ghg_pba[graph_ghg_pba['year'].notna()]
    
    ## Country FPA
    print(' / fpa', end="")
    temp_fpa = temp_fpa.sum(axis=0).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'FPA MtCO2eq'})
    temp_fpa['year'] = year_i
    temp_fpa = pd.merge(graph_countries_data, temp_fpa, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_fpa = pd.concat([graph_ghg_fpa, temp_fpa], ignore_index=True, axis=0, join='outer')
    graph_ghg_fpa = graph_ghg_fpa[graph_ghg_fpa['year'].notna()]
    
    ## Country CBA
    print(' / cba', end="")
    temp_cba = temp_cba.sum(axis=0).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'CBA MtCO2eq'})
    temp_cba['year'] = year_i
    temp_cba = pd.merge(graph_countries_data, temp_cba, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_cba = pd.concat([graph_ghg_cba, temp_cba], ignore_index=True, axis=0, join='outer')
    graph_ghg_cba = graph_ghg_cba[graph_ghg_cba['year'].notna()]
    
    ## Country IM
    print(' / im', end="")
    temp_im = pd.DataFrame(temp_mrio.ghg_diag.D_imp.sort_index().groupby(level='region', axis=0).sum().groupby(level='region', axis=1).sum().sum(axis=0).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'IM MtCO2eq'}))
    temp_im['year'] = year_i
    temp_im = pd.merge(graph_countries_data, temp_im, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_im = pd.concat([graph_ghg_im, temp_im], ignore_index=True, axis=0, join='outer')
    graph_ghg_im = graph_ghg_im[graph_ghg_im['year'].notna()]
    
    ## Country EX
    print(' / ex', end="")
    temp_ex = pd.DataFrame(temp_mrio.ghg_diag.D_imp.sort_index().groupby(level='region', axis=0).sum().groupby(level='region', axis=1).sum().sum(axis=1).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'EX MtCO2eq'}))
    temp_ex['year'] = year_i
    temp_ex = pd.merge(graph_countries_data, temp_ex, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_ex = pd.concat([graph_ghg_ex, temp_ex], ignore_index=True, axis=0, join='outer')
    graph_ghg_ex = graph_ghg_ex[graph_ghg_ex['year'].notna()]
    
    ## Clean Memory
    print(' ... cleanup', end="")
    del temp_mrio
    del temp_gme
    del temp_pba
    del temp_fpa
    del temp_cba
    del temp_im
    del temp_ex
    print(' ... done after: ', str(datetime.timedelta(seconds=round(time.time() - time_start_loop))))


print('Finishing touches ...', end="")
graph_ghg = pd.merge(graph_ghg_pba, graph_ghg_fpa[['iso3', 'country', 'year', 'FPA MtCO2eq']], how='left', on=['iso3', 'country', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
graph_ghg = pd.merge(graph_ghg, graph_ghg_cba[['iso3', 'country', 'year', 'CBA MtCO2eq']], how='left', on=['iso3', 'country', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
graph_ghg = pd.merge(graph_ghg, graph_ghg_im[['iso3', 'country', 'year', 'IM MtCO2eq']], how='left', on=['iso3', 'country', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
graph_ghg = pd.merge(graph_ghg, graph_ghg_ex[['iso3', 'country', 'year', 'EX MtCO2eq']], how='left', on=['iso3', 'country', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
del graph_ghg_pba
del graph_ghg_fpa
del graph_ghg_cba
del graph_ghg_im
del graph_ghg_ex
del graph_countries_data
del graph_countries
print(' done. Total time:', str(datetime.timedelta(seconds=round(time.time() - time_start_all))))

In [ ]:
# Check output for Switzerland
graph_ghg.loc[graph_ghg['iso3'] == 'CHE']

_______________________________________

### Prepare figures dataset

In [ ]:
country_pop = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/wdi pop/wdi_pop_data_reshaped.csv')
graph_ghg = pd.merge(graph_ghg, country_pop, 
                     how='left', 
                     on=['iso3', 'year'], 
                     sort=True,
                     suffixes=('_x', '_y'), 
                     indicator=True, 
                     validate='one_to_one')
graph_ghg = graph_ghg.reset_index()
graph_ghg.set_index(['iso3', 'year'], inplace=True)
del country_pop

In [ ]:
kyoto_year = int('1992')

# Helper Series - Values of CBA MtCO2eq at index 'year' = kyoto_year
s = graph_ghg['CBA MtCO2eq'].xs(kyoto_year, level=1)
# Divide df by helper Series and reset index
graph_ghg['CBA MtCO2eq INDEX'] = 100*graph_ghg['CBA MtCO2eq'].div(s, level=0)

# Helper Series - Values of CBA MtCO2eq at index 'year' = kyoto_year
s = graph_ghg['PBA MtCO2eq'].xs(kyoto_year, level=1)
# Divide df by helper Series
graph_ghg['PBA MtCO2eq INDEX'] = 100*graph_ghg['PBA MtCO2eq'].div(s, level=0)

# CBA p.c. emissions
graph_ghg['CBA tCO2eq p.c.'] = 1000*(graph_ghg['CBA MtCO2eq']) / graph_ghg['population']

# FPA p.c. emissions
graph_ghg['FPA tCO2eq p.c.'] = 1000*(graph_ghg['FPA MtCO2eq']) / graph_ghg['population']

# PBA p.c. emissions
graph_ghg['PBA tCO2eq p.c.'] = 1000*(graph_ghg['PBA MtCO2eq']) / graph_ghg['population']

# IM p.c. emissions
graph_ghg['IM tCO2eq p.c.'] = 1000*(graph_ghg['IM MtCO2eq']) / graph_ghg['population']

# EX p.c. emissions
graph_ghg['EX tCO2eq p.c.'] = 1000*(graph_ghg['EX MtCO2eq']) / graph_ghg['population']

# CBA / PBA ratio
graph_ghg['CBA/PBA'] = (graph_ghg['CBA tCO2eq p.c.']) / graph_ghg['PBA tCO2eq p.c.']

# reset index
graph_ghg = graph_ghg.reset_index()

_______________________________________

### Figure: GHG PBA Indexed

In [ ]:
# initialize necessary data
data = []

## Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['USA','RUS','JPN','DEU','GBR','CAN','FRA','ITA']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 0
rgb_g = 0
rgb_b = 255

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + "," + str(rgb_g) + "," + str(rgb_b) + ")" 

    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append trace
    trace = go.Scatter(x=graph_ghg_select.year.tolist(), 
                       y=graph_ghg_select['PBA MtCO2eq INDEX'].tolist(),
                       mode='lines',
                       line=dict(color= rgb_val),
                       legendgroup="Annex 1",
                       name=group,
                       opacity=0.75,
                       showlegend=False )
    data.append(trace)
    
    # increment color base values
    rgb_g = rgb_g + 30
    rgb_b = rgb_b - 15

    
## NON-Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['CHN','IND','BRA','MEX','KOR','IDN','ZAF','IRN']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 255
rgb_g = 0
rgb_b = 0

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + "," + str(rgb_g) + "," + str(rgb_b) + ")" 
    
    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append trace
    trace = go.Scatter(x=graph_ghg_select.year.tolist(), 
                       y=graph_ghg_select['PBA MtCO2eq INDEX'].tolist(),
                       mode='lines',
                       line=dict(color=rgb_val),
                       legendgroup="Non-Annex 1",
                       name=group,
                       opacity=0.75,
                       showlegend=False )
    data.append(trace)

    # increment color base values
    rgb_g = rgb_g + 20

### generate FIGURE
fig_pba = go.Figure(data=data)

## add Traces for Legend
fig_pba.add_trace(go.Scatter(
    x=['null'],
    y=['null'],
    legendgroup="Annex 1",
    name="<b>Annex B</b>",
    line=dict(color='royalblue'),
    mode='lines'))

fig_pba.add_trace(go.Scatter(
    x=['null'],
    y=['null'],
    legendgroup="Non-Annex 1",
    name="<b>Non-Annex B</b>",
    line=dict(color='red'),
    mode='lines'))


## add Annotations
annotations = []
a = ['CHN','IND','BRA','MEX','KOR','IDN','ZAF','IRN',
     'USA','RUS','JPN','DEU','GBR','CAN','FRA','ITA']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# loop over countries for base annotations
for country, group in groups:
    annotations.append(dict(x=2015, 
                            y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                   & (graph_ghg_select['country'] == country), 
                                                   'PBA MtCO2eq INDEX'].item(), 
                            text=country))

# set base values for all annotations
fig_pba['layout'].update(annotations=annotations)
fig_pba.update_annotations(dict(
            xref="x",
            yref="y",
            showarrow=True,
            arrowhead=7,
            ax=30,
            ay=0,
            xanchor='left'
))

# update annotation y offset
fig_pba.update_annotations(selector=dict(text='India'),patch=dict(ay=-5))
fig_pba.update_annotations(selector=dict(text='Iran'), patch=dict(ay=-5))
fig_pba.update_annotations(selector=dict(text='Indonesia'),patch=dict(ay=5))
fig_pba.update_annotations(selector=dict(text='Mexico'),patch=dict(ay=-7))
fig_pba.update_annotations(selector=dict(text='South Africa'),patch=dict(ay=7))

fig_pba.update_annotations(selector=dict(text='Canada'),patch=dict(ay=-12))
fig_pba.update_annotations(selector=dict(text='Japan'),patch=dict(ay=-18))
fig_pba.update_annotations(selector=dict(text='USA'),patch=dict(ay=-2))
fig_pba.update_annotations(selector=dict(text='Italy'),patch=dict(ay=-13))
fig_pba.update_annotations(selector=dict(text='France'),patch=dict(ay=-2))
fig_pba.update_annotations(selector=dict(text='Russia'),patch=dict(ay=12))
fig_pba.update_annotations(selector=dict(text='Germany'),patch=dict(ay=20))
fig_pba.update_annotations(selector=dict(text='United Kingdom'),patch=dict(ay=14))

# patch USA to United States
fig_pba.update_annotations(selector=dict(text='USA'),patch=dict(text='United States'))

# add UNFCCC vertical line at 1992
fig_pba.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=1992,
            y0=75,
            x1=1992,
            y1=276,
            line=dict(
                color="gray",
                width=1
            )
))
fig_pba.add_annotation(dict(
          y=240,
          x=1991.5,
          align="right",
          valign="top",
          text='UNFCCC adoption',
          showarrow=False,
          xref="x",
          yref="y",
          # Parameter textangle allow you to rotate annotation how you want
          textangle=-90
        ))

# add kyoto vertical line at 1997
fig_pba.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=1997,
            y0=60,
            x1=1997,
            y1=295,
            line=dict(
                color="gray",
                width=1
            )
))
fig_pba.add_annotation(dict(
          y=250,
          x=1996.5,
          align="right",
          valign="top",
          text='Kyoto Protocol adoption',
          showarrow=False,
          xref="x",
          yref="y",
          # Parameter textangle allow you to rotate annotation how you want
          textangle=-90
        ))

## update shapes
fig_pba.update_shapes(dict(xref='x', yref='y'))

## update figure layout
fig_pba.update_layout(template='plotly_white', 
                      font=dict(family="Times",size=16, color='black'),
                      margin=dict(l=75, r=135, t=10, b=10, pad=10),
                      width=1000,
                      height=700,
                      legend=dict(x=0.4, y=-.07),
                      legend_orientation="h")
fig_pba.update_xaxes(range=[1989.75, 2015.25])
fig_pba.update_yaxes(range=[0, 351], dtick = 50)

### show and save figure
fig_pba.show()
fig_pba.write_image("/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_INDEX_PBA.png", scale=10)

### cleanup
del graph_ghg_select
del fig_pba
del data
del a
del trace
del groups
del annotations
del rgb_r
del rgb_g
del rgb_b
del rgb_val

_______________________________________

### Figure:  Subplots Per Capita - v1

In [ ]:
### generate FIGURE
fig_pc = make_subplots(shared_xaxes=True,
                       vertical_spacing=0.05, 
                       horizontal_spacing=0.1, 
                       rows=2, 
                       cols=3,
                       column_titles =('<b>PBA</b><br>tCO<sub>2</sub>-eq p.c.', 
                                       '<b>CBA</b><br>tCO<sub>2</sub>-eq p.c.', 
                                       '<b>CBA / PBA</b><br>ratio'))

## Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['USA','RUS','JPN','DEU','GBR','CAN','FRA','ITA']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 0
rgb_g = 0
rgb_b = 255

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + ',' + str(rgb_g) + ',' + str(rgb_b) + ')' 

    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append traces
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['PBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=1)
    
    fig_pc.add_annotation(text=group,xref='x1',yref='y1',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'PBA tCO2eq p.c.'].item())

    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=1)
    
    fig_pc.add_annotation(text=group,xref='x2',yref='y2',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA/PBA'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=3,row=1)
    
    fig_pc.add_annotation(text=group,xref='x3',yref='y3',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA/PBA'].item())

    
   
    # increment color base values
    rgb_g = rgb_g + 30
    rgb_b = rgb_b - 15

    
## NON-Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['CHN','IND','BRA','MEX','KOR','IDN','ZAF','IRN']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 255
rgb_g = 0
rgb_b = 0

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + ',' + str(rgb_g) + ',' + str(rgb_b) + ')' 
    
    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append traces
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['PBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=2)
    
    fig_pc.add_annotation(text=group,xref='x4',yref='y4',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'PBA tCO2eq p.c.'].item())

    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=2)
    
    fig_pc.add_annotation(text=group,xref='x5',yref='y5',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA/PBA'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=3,row=2)
    
    fig_pc.add_annotation(text=group,xref='x6',yref='y6',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA/PBA'].item())

    # increment color base values
    rgb_g = rgb_g + 20


# set base values for all annotations
fig_pc.update_annotations(
    selector=dict(x=2015),
    patch=dict(showarrow=True,
               arrowhead=7,
               ax=20,
               ay=0,
               xanchor='left'))

# patch USA to United States
fig_pc.update_annotations(selector=dict(text='United Kingdom'),patch=dict(text='UK'))

# update annotation y offset
fig_pc.update_annotations(selector=dict(text='USA'), col=1, row=1, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Canada'), col=1, row=1, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='Germany'), col=1, row=1, patch=dict(ay=-4))
fig_pc.update_annotations(selector=dict(text='Japan'), col=1, row=1, patch=dict(ay=4))
fig_pc.update_annotations(selector=dict(text='UK'), col=1, row=1, patch=dict(ay=3))
fig_pc.update_annotations(selector=dict(text='Italy'), col=1, row=1, patch=dict(ay=10))
fig_pc.update_annotations(selector=dict(text='France'), col=1, row=1, patch=dict(ay=19))

fig_pc.update_annotations(selector=dict(text='Japan'), col=2, row=1, patch=dict(ay=-15))
fig_pc.update_annotations(selector=dict(text='Russia'), col=2, row=1, patch=dict(ay=-7))
fig_pc.update_annotations(selector=dict(text='UK'), col=2, row=1, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='Germany'), col=2, row=1, patch=dict(ay=7))
fig_pc.update_annotations(selector=dict(text='France'), col=2, row=1, patch=dict(ay=8))
fig_pc.update_annotations(selector=dict(text='Italy'), col=2, row=1, patch=dict(ay=16))

fig_pc.update_annotations(selector=dict(text='France'), col=3, row=1, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Italy'), col=3, row=1, patch=dict(ay=1))
fig_pc.update_annotations(selector=dict(text='USA'), col=3, row=1, patch=dict(ay=3))
fig_pc.update_annotations(selector=dict(text='Germany'), col=3, row=1, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='Canada'), col=3, row=1, patch=dict(ay=15))
fig_pc.update_annotations(selector=dict(text='Russia'), col=3, row=1, patch=dict(ay=7))

fig_pc.update_annotations(selector=dict(text='South Korea'), col=1, row=2, patch=dict(ay=-7))
fig_pc.update_annotations(selector=dict(text='Iran'), col=1, row=2, patch=dict(ay=-12))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=1, row=2, patch=dict(ay=-9))
fig_pc.update_annotations(selector=dict(text='China'), col=1, row=2, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=1, row=2, patch=dict(ay=-9))
fig_pc.update_annotations(selector=dict(text='Brazil'), col=1, row=2, patch=dict(ay=0))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=1, row=2, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='India'), col=1, row=2, patch=dict(ay=3))

fig_pc.update_annotations(selector=dict(text='South Korea'), col=2, row=2, patch=dict(ay=-12))
fig_pc.update_annotations(selector=dict(text='Iran'), col=2, row=2, patch=dict(ay=-18))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=2, row=2, patch=dict(ay=-19))
fig_pc.update_annotations(selector=dict(text='China'), col=2, row=2, patch=dict(ay=-9))
fig_pc.update_annotations(selector=dict(text='Brazil'), col=2, row=2, patch=dict(ay=-11))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=2, row=2, patch=dict(ay=-2))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=2, row=2, patch=dict(ay=-3))
fig_pc.update_annotations(selector=dict(text='India'), col=2, row=2, patch=dict(ay=2))

fig_pc.update_annotations(selector=dict(text='Brazil'), col=3, row=2, patch=dict(ay=-32))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=3, row=2, patch=dict(ay=-24))
fig_pc.update_annotations(selector=dict(text='South Korea'), col=3, row=2, patch=dict(ay=-14))
fig_pc.update_annotations(selector=dict(text='Iran'), col=3, row=2, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=3, row=2, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='India'), col=3, row=2, patch=dict(ay=15))
fig_pc.update_annotations(selector=dict(text='China'), col=3, row=2, patch=dict(ay=27))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=3, row=2, patch=dict(ay=35))

## update figure layout
fig_pc.update_layout(template='plotly_white', 
                     font=dict(family='Times',size=12, color='black'),
                     margin=dict(l=100, r=100, t=75, b=10, pad=5),
                     width=1300, height=700,)
fig_pc.update_xaxes(range=[1989.5, 2015.5], dtick=5)
fig_pc.update_yaxes(range=[0, 41], dtick = 10)
fig_pc.update_yaxes(range=[0, 2.05], dtick=0.5, col=3)

fig_pc.update_yaxes(title_text='<b>Annex 1</b>', row=1, col=1, titlefont = dict(size = 16))
fig_pc.update_yaxes(title_text='<b>Non-Annex 1</b>', row=2, col=1, titlefont = dict(size = 16))

### show and save figure
fig_pc.show()
fig_pc.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_PC.png', scale=10)

### cleanup
del graph_ghg_select
del fig_pc
del a
del groups
del rgb_r
del rgb_g
del rgb_b
del rgb_val

_______________________________________

### Figure:  Subplots Per Capita - v2

In [ ]:
### generate FIGURE
fig_pc = make_subplots(shared_xaxes=True,
                       vertical_spacing=0.05, 
                       horizontal_spacing=0.15, 
                       rows=4, 
                       cols=2,
                       column_titles =('<b>Annex B</b><br> ',
                                       '<b>Non-Annex B</b><br> '))

## Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['USA','RUS','JPN','DEU','GBR','CAN','FRA','ITA']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 0
rgb_g = 0
rgb_b = 255

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + ',' + str(rgb_g) + ',' + str(rgb_b) + ')' 

    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append traces
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['PBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=1)
    
    fig_pc.add_annotation(text=group,xref='x1',yref='y1',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'PBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['FPA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=2)
    
    fig_pc.add_annotation(text=group,xref='x3',yref='y3',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'FPA tCO2eq p.c.'].item())

    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=3)
    
    fig_pc.add_annotation(text=group,xref='x5',yref='y5',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA/PBA'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=1,row=4)
    
    fig_pc.add_annotation(text=group,xref='x7',yref='y7',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA/PBA'].item())

    
   
    # increment color base values
    rgb_g = rgb_g + 30
    rgb_b = rgb_b - 15

    
## NON-Annex 1 Country Traces
# reduce dataframe to desired countries
a = ['CHN','IND','BRA','MEX','KOR','IDN','ZAF','IRN']
graph_ghg_select = graph_ghg[graph_ghg['iso3'].isin(a)]
groups = graph_ghg_select.groupby(by='country')

# set starting colors
rgb_r = 255
rgb_g = 0
rgb_b = 0

# loop over countries to generate traces
for group, graph_ghg_select in groups:
    
    # get color value
    rgb_val = 'rgb(' + str(rgb_r) + ',' + str(rgb_g) + ',' + str(rgb_b) + ')' 
    
    # sort dataframe
    graph_ghg_select = graph_ghg_select.sort_values(by=['year'])
    
    # build and append traces
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['PBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=1)
    
    fig_pc.add_annotation(text=group,xref='x2',yref='y2',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'PBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['FPA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=2)
    
    fig_pc.add_annotation(text=group,xref='x4',yref='y4',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'FPA tCO2eq p.c.'].item())

    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA tCO2eq p.c.'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=3)
    
    fig_pc.add_annotation(text=group,xref='x6',yref='y6',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA tCO2eq p.c.'].item())
    
    fig_pc.append_trace(go.Scatter(x=graph_ghg_select.year.tolist(), 
                                   y=graph_ghg_select['CBA/PBA'].tolist(),
                                   mode='lines', line=dict(color= rgb_val), opacity=0.75,
                                   legendgroup='Annex 1', name=group, showlegend=False),
                        col=2,row=4)
    
    fig_pc.add_annotation(text=group,xref='x8',yref='y8',
                          x=2015, y=graph_ghg_select.loc[(graph_ghg_select['year'] == 2015) 
                                                        &(graph_ghg_select['country'] == group), 
                                                        'CBA/PBA'].item())

    # increment color base values
    rgb_g = rgb_g + 20


# set base values for all annotations
fig_pc.update_annotations(
    selector=dict(x=2015),
    patch=dict(showarrow=True,
               arrowhead=7,
               ax=20,
               ay=0,
               xanchor='left'))

# patch USA to United States
fig_pc.update_annotations(selector=dict(text='United Kingdom'),patch=dict(text='UK'))

# update annotation y offset
fig_pc.update_annotations(selector=dict(text='USA'), col=1, row=1, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Canada'), col=1, row=1, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='Germany'), col=1, row=1, patch=dict(ay=-4))
fig_pc.update_annotations(selector=dict(text='Japan'), col=1, row=1, patch=dict(ay=4))
fig_pc.update_annotations(selector=dict(text='UK'), col=1, row=1, patch=dict(ay=3))
fig_pc.update_annotations(selector=dict(text='Italy'), col=1, row=1, patch=dict(ay=10))
fig_pc.update_annotations(selector=dict(text='France'), col=1, row=1, patch=dict(ay=19))

fig_pc.update_annotations(selector=dict(text='USA'), col=1, row=2, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Japan'), col=1, row=2, patch=dict(ay=-17))
fig_pc.update_annotations(selector=dict(text='Russia'), col=1, row=2, patch=dict(ay=2))
fig_pc.update_annotations(selector=dict(text='UK'), col=1, row=2, patch=dict(ay=-8))
fig_pc.update_annotations(selector=dict(text='Germany'), col=1, row=2, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='France'), col=1, row=2, patch=dict(ay=8))
fig_pc.update_annotations(selector=dict(text='Italy'), col=1, row=2, patch=dict(ay=16))

fig_pc.update_annotations(selector=dict(text='Japan'), col=1, row=3, patch=dict(ay=-15))
fig_pc.update_annotations(selector=dict(text='Russia'), col=1, row=3, patch=dict(ay=-7))
fig_pc.update_annotations(selector=dict(text='UK'), col=1, row=3, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='Germany'), col=1, row=3, patch=dict(ay=7))
fig_pc.update_annotations(selector=dict(text='France'), col=1, row=3, patch=dict(ay=8))
fig_pc.update_annotations(selector=dict(text='Italy'), col=1, row=3, patch=dict(ay=16))

fig_pc.update_annotations(selector=dict(text='France'), col=1, row=4, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Italy'), col=1, row=4, patch=dict(ay=1))
fig_pc.update_annotations(selector=dict(text='USA'), col=1, row=4, patch=dict(ay=3))
fig_pc.update_annotations(selector=dict(text='Germany'), col=1, row=4, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='Canada'), col=1, row=4, patch=dict(ay=15))
fig_pc.update_annotations(selector=dict(text='Russia'), col=1, row=4, patch=dict(ay=7))

fig_pc.update_annotations(selector=dict(text='South Korea'), col=2, row=1, patch=dict(ay=-7))
fig_pc.update_annotations(selector=dict(text='Iran'),col=2, row=1, patch=dict(ay=-12))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=2, row=1, patch=dict(ay=-9))
fig_pc.update_annotations(selector=dict(text='China'), col=2, row=1, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=2, row=1, patch=dict(ay=-9))
fig_pc.update_annotations(selector=dict(text='Brazil'), col=2, row=1, patch=dict(ay=0))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=2, row=1, patch=dict(ay=-1))
fig_pc.update_annotations(selector=dict(text='India'), col=2, row=1, patch=dict(ay=3))

fig_pc.update_annotations(selector=dict(text='South Korea'), col=2, row=2, patch=dict(ay=-12))
fig_pc.update_annotations(selector=dict(text='Iran'), col=2, row=2, patch=dict(ay=-21))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=2, row=2, patch=dict(ay=-14))
fig_pc.update_annotations(selector=dict(text='China'), col=2, row=2, patch=dict(ay=-7))
fig_pc.update_annotations(selector=dict(text='Brazil'), col=2, row=2, patch=dict(ay=0))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=2, row=2, patch=dict(ay=-10))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=2, row=2, patch=dict(ay=-3))
fig_pc.update_annotations(selector=dict(text='India'), col=2, row=2, patch=dict(ay=2))

fig_pc.update_annotations(selector=dict(text='South Korea'), col=2, row=3, patch=dict(ay=-12))
fig_pc.update_annotations(selector=dict(text='Iran'), col=2, row=3, patch=dict(ay=-19))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=2, row=3, patch=dict(ay=-19))
fig_pc.update_annotations(selector=dict(text='China'), col=2, row=3, patch=dict(ay=-8))
fig_pc.update_annotations(selector=dict(text='Brazil'), col=2, row=3, patch=dict(ay=-11))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=2, row=3, patch=dict(ay=-2))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=2, row=3, patch=dict(ay=-3))
fig_pc.update_annotations(selector=dict(text='India'), col=2, row=3, patch=dict(ay=2))

fig_pc.update_annotations(selector=dict(text='Brazil'), col=2, row=4, patch=dict(ay=-32))
fig_pc.update_annotations(selector=dict(text='Mexico'), col=2, row=4, patch=dict(ay=-24))
fig_pc.update_annotations(selector=dict(text='South Korea'), col=2, row=4, patch=dict(ay=-14))
fig_pc.update_annotations(selector=dict(text='Iran'), col=2, row=4, patch=dict(ay=-5))
fig_pc.update_annotations(selector=dict(text='Indonesia'), col=2, row=4, patch=dict(ay=5))
fig_pc.update_annotations(selector=dict(text='India'), col=2, row=4, patch=dict(ay=15))
fig_pc.update_annotations(selector=dict(text='China'), col=2, row=4, patch=dict(ay=27))
fig_pc.update_annotations(selector=dict(text='South Africa'), col=2, row=4, patch=dict(ay=35))

## update figure layout
fig_pc.update_layout(template='plotly_white', 
                     font=dict(family='Times',size=12, color='black'),
                     margin=dict(l=125, r=125, t=75, b=75, pad=10),
                     width=1000, height=1250,)
fig_pc.update_xaxes(range=[1989.5, 2015.5], dtick=5)
fig_pc.update_yaxes(range=[0, 41], dtick = 10)
#fig_pc.update_yaxes(range=[0, 51], dtick = 10, row=2)
fig_pc.update_yaxes(range=[0, 2.05], dtick=0.5, row=4)

fig_pc.update_yaxes(title_text='<b>PBA</b><br>tCO<sub>2</sub>-eq p.c.', row=1, col=1, titlefont = dict(size = 16))
fig_pc.update_yaxes(title_text='<b>FPA</b><br>tCO<sub>2</sub>-eq p.c.', row=2, col=1, titlefont = dict(size = 16))
fig_pc.update_yaxes(title_text='<b>CBA</b><br>tCO<sub>2</sub>-eq p.c.', row=3, col=1, titlefont = dict(size = 16))
fig_pc.update_yaxes(title_text='<b>CBA / PBA</b><br>ratio'            , row=4, col=1, titlefont = dict(size = 16))

### show and save figure
fig_pc.show()
fig_pc.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_PC_2.png', scale=10)

### cleanup
del graph_ghg_select
del fig_pc
del a
del groups
del rgb_r
del rgb_g
del rgb_b
del rgb_val

_______________________________________

### Figure:  Flows between Annex Groups

In [ ]:
graph_ghg_flows = data_gme[['iso3_o','iso3_d','ghg_co2eq','year']]

annex_yes = ['AUS',	'AUT',	'BEL',	'BGR',	'CAN',	'CHE',	'CZE',	'DEU',	'DNK',	'ESP',	'EST',	'FIN',	'FRA',	'GBR',	'GRC',	'HRV',	'HUN',	'IRL',	'ISL',	'ITA',	'JPN',	'LIE',	'LTU',	'LUX',	'LVA',	'MCO',	'MLT',	'NLD',	'NOR',	'NZL',	'POL',	'PRT',	'RUS',	'SVK',	'SVN',	'SWE',	'UKR',	'USA']
annex_no  = ['ABW',	'AFG',	'AGO',	'ALB',	'AND',	'ARE',	'ARG',	'ARM',	'ATG',	'AZE',	'BDI',	'BEN',	'BFA',	'BGD',	'BHR',	'BHS',	'BIH',	'BLR',	'BLZ',	'BOL',	'BRA',	'BRB',	'BRN',	'BTN',	'BWA',	'CAF',	'CHL',	'CHN',	'CIV',	'CMR',	'COD',	'COG',	'COL',	'CPV',	'CRI',	'CUB',	'CYP',	'DJI',	'DOM',	'DZA',	'ECU',	'EGY',	'ERI',	'ETH',	'FJI',	'GAB',	'GEO',	'GHA',	'GIN',	'GMB',	'GTM',	'GUY',	'HND',	'HTI',	'IDN',	'IND',	'IRN',	'IRQ',	'ISR',	'JAM',	'JOR',	'KAZ',	'KEN',	'KGZ',	'KHM',	'KOR',	'KWT',	'LAO',	'LBN',	'LBR',	'LBY',	'LKA',	'LSO',	'MAC',	'MAR',	'MDA',	'MDG',	'MDV',	'MEX',	'MKD',	'MLI',	'MMR',	'MNG',	'MOZ',	'MRT',	'MUS',	'MWI',	'MYS',	'NAM',	'NER',	'NGA',	'NIC',	'NPL',	'OMN',	'PAK',	'PAN',	'PER',	'PHL',	'PNG',	'PRK',	'PRY',	'QAT',	'RWA',	'SAU',	'SEN',	'SGP',	'SLE',	'SLV',	'SMR',	'SOM',	'STP',	'SUD',	'SUR',	'SWZ',	'SYC',	'SYR',	'TCD',	'TGO',	'THA',	'TJK',	'TKM',	'TUR',	'TTO',	'TUN',	'TZA',	'UGA',	'URY',	'UZB',	'VEN',	'VGB',	'VNM',	'VUT',	'WSM',	'YEM',	'ZAF',	'ZMB',	'ZWE',]

graph_ghg_flows['grp_o'] = graph_ghg_flows['iso3_o'].replace(annex_yes, 'Annex 1').replace(annex_no, 'Non-Annex 1')
graph_ghg_flows['grp_d'] = graph_ghg_flows['iso3_d'].replace(annex_yes, 'Annex 1').replace(annex_no, 'Non-Annex 1')
graph_ghg_flows = graph_ghg_flows[graph_ghg_flows['grp_o'].isin(['Annex 1', 'Non-Annex 1'])]
graph_ghg_flows = graph_ghg_flows[graph_ghg_flows['grp_d'].isin(['Annex 1', 'Non-Annex 1'])]
graph_ghg_flows = graph_ghg_flows[['grp_o','grp_d','ghg_co2eq','year']]
graph_ghg_flows = graph_ghg_flows.groupby(['grp_o','grp_d','year']).sum().sort_values(by=['grp_o','grp_d','year']).reset_index()
graph_ghg_flows['ghg_co2eq'] = graph_ghg_flows['ghg_co2eq'] / 1000000

### generate FIGURE
fig_flows = make_subplots(shared_xaxes=True,
                           vertical_spacing=0.3, 
                           horizontal_spacing=0.1, 
                           rows=2, 
                           cols=2,
                           subplot_titles =('<b>Within Annex B countries</b><br>trade embodied CO<sub>2</sub>-eq flows (Gt)', 
                                            '<b>Within Non-Annex B countries</b><br>trade embodied CO<sub>2</sub>-eq flows (Gt)', 
                                            '<b>Between Annex B and Non-Annex B countries</b><br>trade embodied CO<sub>2</sub>-eq flows (Gt)'),
                           specs=[[{}            ,{}   ],
                                  [{'colspan': 2}, None]])

fig_flows.append_trace(go.Scatter(x=graph_ghg_flows['year']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Annex 1')].tolist(), 
                                  y=graph_ghg_flows['ghg_co2eq']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Annex 1')].tolist(),
                               mode='lines', line=dict(color= 'royalblue'), opacity=0.75,
                               legendgroup='Annex 1', name='Annex 1', showlegend=False),
                        col=1,row=1)

fig_flows.append_trace(go.Scatter(x=graph_ghg_flows['year']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Non-Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Non-Annex 1')].tolist(), 
                                  y=graph_ghg_flows['ghg_co2eq']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Non-Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Non-Annex 1')].tolist(),
                               mode='lines', line=dict(color= 'red'), opacity=0.75,
                               legendgroup='Non-Annex 1', name='Non-Annex 1', showlegend=False),
                        col=2,row=1)

fig_flows.append_trace(go.Scatter(x=graph_ghg_flows['year']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Non-Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Annex 1')].tolist(), 
                                  y=graph_ghg_flows['ghg_co2eq']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Non-Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Annex 1')].tolist(),
                               mode='lines', line=dict(color= 'red'), opacity=0.75,
                               legendgroup='Non-Annex 1', name='Non-Annex 1', showlegend=False),
                        col=1,row=2)

fig_flows.append_trace(go.Scatter(x=graph_ghg_flows['year']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Non-Annex 1')].tolist(), 
                                  y=graph_ghg_flows['ghg_co2eq']
                                  .loc[(graph_ghg_flows['grp_o'] == 'Annex 1') 
                                      &(graph_ghg_flows['grp_d'] == 'Non-Annex 1')].tolist(),
                               mode='lines', line=dict(color= 'royalblue'), opacity=0.75,
                               legendgroup='Annex 1', name='Annex 1', showlegend=False),
                        col=1,row=2)

## add Traces for Legend
fig_flows.add_trace(go.Scatter(
    x=['null'],
    y=['null'],
    legendgroup="Annex 1",
    name="<b>Annex B exports </b>",
    line=dict(color='royalblue'),
    mode='lines'))

fig_flows.add_trace(go.Scatter(
    x=['null'],
    y=['null'],
    legendgroup="Non-Annex 1",
    name="<b>Non-Annex B exports</b>",
    line=dict(color='red'),
    mode='lines'))

## update figure layout
fig_flows.update_layout(template='plotly_white', 
                         font=dict(family='Times',size=16, color='black'),
                         margin=dict(l=75, r=75, t=75, b=10, pad=10),
                         width=1000, height=600,
                         legend=dict(x=0.28, y=-.1, font=dict(size=16)),
                         legend_orientation="h")

fig_flows.update_xaxes(range=[1989.9, 2015.1], row=1)
fig_flows.update_xaxes(range=[1989.95, 2015.05], row=2)

fig_flows.update_yaxes(range=[0, 30.5], dtick = 5)
fig_flows.update_yaxes(range=[0, 5.1], dtick = 1, row=2)

fig_flows.show()

fig_flows.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_FLOWS.png', scale=10)

### cleanup
del graph_ghg_flows
del fig_flows
del annex_yes
del annex_no

_______________________________________

### Figure:  Chloropleth World Maps

In [ ]:
world_subplots_data_1997 = graph_ghg[graph_ghg['year'] == 1997]
world_subplots_data_2015 = graph_ghg[graph_ghg['year'] == 2015]


world_subplots = make_subplots( rows=3, cols=2,
                                specs=[[{"type": "choropleth"}, {"type": "choropleth"}],
                                       [{"type": "choropleth"}, {"type": "choropleth"}],
                                        [{"type": "choropleth"}, {"type": "choropleth"}]],
                                column_titles =('<b>1997</b>','<b>2015</b>'),
                                row_titles =('<b>PBA</b>','<b>FPA</b>','<b>CBA</b>'),
                                vertical_spacing=0.001, 
                                horizontal_spacing=0.0 )

map_colorscale = px.colors.sequential.Jet
map_colorbar_thickness = 7.5
map_colorbar_title = dict(text=' <br><b>tCO<sub>2</sub>eq p.c.<b>', side = 'right')
map_reversescale = False
map_marker = dict(line=dict(width=0))
map_zmin = 0
map_zmax = 25

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_1997['iso3'],
                                z = world_subplots_data_1997['PBA tCO2eq p.c.'],
                                text = world_subplots_data_1997['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker,
                                showscale=False 
                                ), col=1,row=1)

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_1997['iso3'],
                                z = world_subplots_data_1997['FPA tCO2eq p.c.'],
                                text = world_subplots_data_1997['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker,
                                showscale=False 
                                ), col=1,row=2)

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_1997['iso3'],
                                z = world_subplots_data_1997['CBA tCO2eq p.c.'],
                                text = world_subplots_data_1997['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker,
                                showscale=False 
                                ), col=1,row=3)

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_2015['iso3'],
                                z = world_subplots_data_2015['PBA tCO2eq p.c.'],
                                text = world_subplots_data_2015['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker ,
                                showscale=False    
                                ), col=2,row=1)

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_2015['iso3'],
                                z = world_subplots_data_2015['FPA tCO2eq p.c.'],
                                text = world_subplots_data_2015['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker,
                                showscale=False 
                                ), col=2,row=2)

world_subplots.append_trace(go.Choropleth(
                                locations = world_subplots_data_2015['iso3'],
                                z = world_subplots_data_2015['CBA tCO2eq p.c.'],
                                text = world_subplots_data_2015['country'],
                                colorbar_title = map_colorbar_title,
                                colorbar_thickness = map_colorbar_thickness,
                                zmin = map_zmin,
                                zmax = map_zmax,
                                reversescale = map_reversescale,
                                colorscale=map_colorscale,
                                marker = map_marker,
                                colorbar_ticksuffix = '+',
                                colorbar_showticksuffix = 'last',
                                ), col=2,row=3)

geodict = dict(
        landcolor = 'lightgray',
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth',
        projection_rotation=dict(lon=10.5),
        lataxis_range=[-60,90],
        resolution = 50
    )

world_subplots.update_annotations(selector=dict(text='<b>PBA</b>'), patch=dict(x= -0.02, textangle = -90),font=dict(family='Times',size=20, color='black'))
world_subplots.update_annotations(selector=dict(text='<b>FPA</b>'), patch=dict(x= -0.02, textangle = -90),font=dict(family='Times',size=20, color='black'))
world_subplots.update_annotations(selector=dict(text='<b>CBA</b>'), patch=dict(x= -0.02, textangle = -90),font=dict(family='Times',size=20, color='black'))
world_subplots.update_annotations(selector=dict(text='<b>1997</b>'), font=dict(family='Times',size=20, color='black'))
world_subplots.update_annotations(selector=dict(text='<b>2015</b>'), font=dict(family='Times',size=20, color='black'))
   

world_subplots.update_layout(geo = geodict,
                             geo2= geodict,
                             geo3= geodict,
                             geo4= geodict,
                             geo5= geodict,
                             geo6= geodict,
                             template='plotly_white', 
                             font=dict(family='Times',size=16, color='black'),
                             margin=dict(l=75, r=75, t=75, b=10, pad=10),
                             width=1100, height=750)

world_subplots.show()
world_subplots.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_World_Trends.png', scale=10)

del world_subplots
del world_subplots_data_1997
del world_subplots_data_2015
del map_colorscale
del map_colorbar_thickness
del map_colorbar_title
del map_reversescale
del map_marker
del map_zmin
del map_zmax
del geodict

_______________________________________

### Figures:  Country Graphs 

In [ ]:
print('Generating graphs .', end="")


for iso3 in graph_ghg.iso3.unique():
#for iso3 in ['CHE']:

    print('.. ' + iso3 +' ', end="")
    
    # Select data
    temp_data = graph_ghg[graph_ghg['iso3'] == iso3]

    # Init graph
    temp_country_graph = go.Figure()

    # Add traces
    temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['PBA tCO2eq p.c.'],
                                mode='lines', line=dict(color= 'black', width=2),
                                name='<b>PBA</b>'))
    temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['FPA tCO2eq p.c.'],
                                mode='lines', line=dict(color= 'red', width=2, dash='dot'),
                                name='<b>FPA</b>'))
    temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['CBA tCO2eq p.c.'],
                                mode='lines', line=dict(color= 'blue', width=2, dash='dot'),
                                name='<b>CBA</b>'))
    
    temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['IM tCO2eq p.c.'],
                                mode='lines', line=dict(color= 'darkorange', width=1, dash='dash'),
                                name='<b>Imports</b>'))
    temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['EX tCO2eq p.c.'],
                                mode='lines', line=dict(color= 'limegreen', width=1, dash='dash'),
                                name='<b>Exports</b>'))
    
    ## Shapes
    # UNFCCC at 1992
    temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                      xref = 'x'    , x0 = 1992 , x1=1992, 
                                      yref = 'paper', y0 = 0 , y1=1.2))
    temp_country_graph.add_annotation(dict(text = '<b>UNFCCC</b>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = 1992,  align = 'left', xanchor = 'left',
                                           yref = 'paper', y = 1.20, valign = 'top'))

    # Kyoto Protocol at 1997
    temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                      xref = 'x'    , x0 = 1997 , x1=1997, 
                                      yref = 'paper', y0 = 0 , y1=1.2))
    temp_country_graph.add_annotation(dict(text = '<b>Kyoto Protocol</b>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = 1997,  align = 'left', xanchor = 'left',
                                           yref = 'paper', y = 1.20, valign = 'top'))
    # Country dates
    d_si = (temp_data['kyoto_date_sign'].unique()[0].astype('datetime64[Y]').astype(int) + 1970)+((temp_data['kyoto_date_sign'].unique()[0].astype('datetime64[M]').astype(int) % 12 + 1) / 12 )
    d_ra = (temp_data['kyoto_date_ratify'].unique()[0].astype('datetime64[Y]').astype(int) + 1970)+((temp_data['kyoto_date_ratify'].unique()[0].astype('datetime64[M]').astype(int) % 12 + 1) / 12 )
    d_le = (temp_data['kyoto_date_leave'].unique()[0].astype('datetime64[Y]').astype(int) + 1970)+(((temp_data['kyoto_date_leave'].unique()[0].astype('datetime64[M]').astype(int) % 12 + 1) / 12 )-0.05)

    # Kyoto Protocol signed
    temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                      xref = 'x'    , x0 = d_si , x1=d_si, 
                                      yref = 'paper', y0 = 0 , y1=1.1))
    temp_country_graph.add_annotation(dict(text = '<i>signed</i>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = d_si,  align = 'left', xanchor = 'left',
                                           yref = 'paper', y = 1.1, valign = 'top'))
        
    
    # Kyoto Protocol ratified
    temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                      xref = 'x'    , x0 = d_ra , x1=d_ra, 
                                      yref = 'paper', y0 = 0 , y1=1.1))
    temp_country_graph.add_annotation(dict(text = '<i>ratified</i>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = d_ra,  align = 'left', xanchor = 'left',
                                           yref = 'paper', y = 1.1, valign = 'top'))
    # fix for overlapping annotations
    if (d_ra - d_si) <=2:
        temp_country_graph.update_annotations(selector=dict(text='<i>ratified</i>'), patch=dict(y = 1.075))
        temp_country_graph.update_shapes(selector=dict(x1=d_ra)  , patch=dict(y1 = 1.07))
        temp_country_graph.update_annotations(selector=dict(text='<i>signed</i>')  , patch=dict(y = 1.14))
        temp_country_graph.update_shapes(selector=dict(x1=d_si)  , patch=dict(y1 = 1.135))


    # Kyoto Protocol left
    temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                      xref = 'x'    , x0 = d_le , x1=d_le, 
                                      yref = 'paper', y0 = 0 , y1=1.1))
    temp_country_graph.add_annotation(dict(text = '<i>withdrawn</i>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = d_le,  align = 'left', xanchor = 'left',
                                           yref = 'paper', y = 1.1, valign = 'top'))

    # add commitment period 
    if temp_data['kyoto_annex_1'].unique()[0] == 'Annex I':
        temp_country_graph.add_shape(dict(type = 'rect' , fillcolor="lightgrey", opacity=0.5, layer="below", line_width=0,
                                      xref = 'x'    , x0 = 2008 , x1=2012.99, 
                                      yref = 'paper', y0 = 0 , y1=1.2))
        temp_country_graph.add_annotation(dict(text = '<b>Period 1</b>', font=dict(size = 10), showarrow = False,
                                               xref = 'x'    , x = 2010.5,  align = 'center',
                                               yref = 'paper', y = 1.2, valign = 'top'))


    ## set axis tatle
    temp_country_graph.update_yaxes(title_text='<b>tCO<sub>2</sub>-eq p.c. </b>')

    ## update figure layout
    temp_country_graph.update_layout( title={'text': '<b>'+temp_data['country'].unique()[0]+'</b> - Emissions Trends',
                                             'x':0.5,
                                             'y':0.95,
                                             'font': dict(size=20),
                                             'xanchor': 'center',
                                             'yanchor': 'top'},
                                      yaxis_title="<b>tCO<sub>2</sub>-eq p.c.</b>",template='plotly_white', 
                                      font=dict(family="Times",size=12, color='black'),
                                      margin=dict(l=75, r=25, t=100, b=50, pad=5),
                                      width=500, height=400,
                                      legend=dict(x=0.02, y=-0.1),
                                      legend_orientation="h")
    
    ## set range conditions
    temp_country_graph.update_xaxes(range=[1989.75, 2015.25], showgrid=False)
        
    if 10 > max(temp_data[['PBA tCO2eq p.c.','FPA tCO2eq p.c.','CBA tCO2eq p.c.']].max()):
        temp_country_graph.update_yaxes(range=[0, 10.1], dtick = 2)
    elif 50 < max(temp_data[['PBA tCO2eq p.c.','FPA tCO2eq p.c.','CBA tCO2eq p.c.']].max()) < 100:
        temp_country_graph.update_yaxes(rangemode = 'tozero', dtick = 10)
    elif 100 < max(temp_data[['PBA tCO2eq p.c.','FPA tCO2eq p.c.','CBA tCO2eq p.c.']].max()):
        temp_country_graph.update_yaxes(rangemode = 'tozero', dtick = 25)
    else :
        temp_country_graph.update_yaxes(range=[0, 50.5], dtick = 10)

    ##  Show & Save graph
    #temp_country_graph.show()
    temp_country_graph.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/Countries/GHG_'+ iso3 + '.png', scale=10)

    # cleanup
    del temp_country_graph
    del temp_data
    del d_si
    del d_ra
    del d_le
    
print('... done.')

_______________________________________

### Figure: Annex B Trends

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
temp_data2 = graph_ghg[['kyoto_annex_1','year','PBA MtCO2eq','FPA MtCO2eq','CBA MtCO2eq']]
temp_data2 = temp_data2.groupby(['kyoto_annex_1', 'year']).agg({'PBA MtCO2eq':'sum','FPA MtCO2eq':'sum','CBA MtCO2eq':'sum'}).reset_index()

temp_data2['PBA GtCO2eq'] = ((temp_data2['PBA MtCO2eq'])/1000000)
temp_data2['FPA GtCO2eq'] = ((temp_data2['FPA MtCO2eq'])/1000000)
temp_data2['CBA GtCO2eq'] = ((temp_data2['CBA MtCO2eq'])/1000000)
temp_data2['Net Imports'] = ((temp_data2['CBA GtCO2eq'] - temp_data2['PBA GtCO2eq']))

In [ ]:
# Select data
temp_data = temp_data2[temp_data2['kyoto_annex_1'] == 'Annex I']

# Init graph
temp_country_graph = go.Figure()

# Add traces
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['PBA GtCO2eq'],
                            mode='lines', line=dict(color= 'black', width=2),
                            name='<b>PBA</b>'))
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['FPA GtCO2eq'],
                            mode='lines', line=dict(color= 'red', width=2, dash='dot'),
                            name='<b>FPA</b>'))
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['CBA GtCO2eq'],
                            mode='lines', line=dict(color= 'blue', width=2, dash='dot'),
                            name='<b>CBA</b>'))

## Shapes
# UNFCCC at 1992
temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                  xref = 'x'    , x0 = 1992 , x1=1992, 
                                  yref = 'paper', y0 = 0 , y1=1.2))
temp_country_graph.add_annotation(dict(text = '<b>UNFCCC</b>', font=dict(size = 10), showarrow = False,
                                       xref = 'x'    , x = 1992,  align = 'left', xanchor = 'left',
                                       yref = 'paper', y = 1.20, valign = 'top'))

# Kyoto Protocol at 1997
temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                  xref = 'x'    , x0 = 1997 , x1=1997, 
                                  yref = 'paper', y0 = 0 , y1=1.2))
temp_country_graph.add_annotation(dict(text = '<b>Kyoto Protocol</b>', font=dict(size = 10), showarrow = False,
                                       xref = 'x'    , x = 1997,  align = 'left', xanchor = 'left',
                                       yref = 'paper', y = 1.20, valign = 'top'))

# add commitment period 
if temp_data['kyoto_annex_1'].unique()[0] == 'Annex I':
    temp_country_graph.add_shape(dict(type = 'rect' , fillcolor="lightgrey", opacity=0.5, layer="below", line_width=0,
                                      xref = 'x'    , x0 = 2008 , x1=2012.99, 
                                      yref = 'paper', y0 = 0 , y1=1.2))
    temp_country_graph.add_annotation(dict(text = '<b>Period 1</b>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = 2010.5,  align = 'center',
                                           yref = 'paper', y = 1.2, valign = 'top'))


## set axis tatle
temp_country_graph.update_yaxes(title_text='<b>GtCO<sub>2</sub>-eq</b>')

## update figure layout
temp_country_graph.update_layout( title={'text': 'Annex B - Emissions Trends',
                                         'x':0.5,
                                         'y':0.95,
                                         'font': dict(size=20),
                                         'xanchor': 'center',
                                         'yanchor': 'top'},
                                  yaxis_title="<b>GtCO<sub>2</sub>-eq</b>",template='plotly_white', 
                                  font=dict(family="Times",size=12, color='black'),
                                  margin=dict(l=75, r=25, t=100, b=50, pad=5),
                                  width=500, height=400,
                                  legend=dict(x=0.2, y=-0.1),
                                  legend_orientation="h")

## set range conditions
temp_country_graph.update_xaxes(range=[1989.75, 2015.25], showgrid=False)
    
temp_country_graph.update_yaxes(range=[0, 50.5], dtick = 10)

##  Show & Save graph
temp_country_graph.show()
temp_country_graph.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_Annex-B.png', scale=10)

# cleanup
del temp_country_graph
del temp_data

_______________________________________

### Figure: Non-Annex B Trends

In [ ]:
# Select data
temp_data = temp_data2[temp_data2['kyoto_annex_1'] == 'Non-Annex I']

# Init graph
temp_country_graph = go.Figure()

# Add traces
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['PBA GtCO2eq'],
                            mode='lines', line=dict(color= 'black', width=2),
                            name='<b>PBA</b>'))
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['FPA GtCO2eq'],
                            mode='lines', line=dict(color= 'red', width=2, dash='dot'),
                            name='<b>FPA</b>'))
temp_country_graph.add_trace(go.Scatter(x=temp_data['year'], y=temp_data['CBA GtCO2eq'],
                            mode='lines', line=dict(color= 'blue', width=2, dash='dot'),
                            name='<b>CBA</b>'))
    
## Shapes
# UNFCCC at 1992
temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                  xref = 'x'    , x0 = 1992 , x1=1992, 
                                  yref = 'paper', y0 = 0 , y1=1.2))
temp_country_graph.add_annotation(dict(text = '<b>UNFCCC</b>', font=dict(size = 10), showarrow = False,
                                       xref = 'x'    , x = 1992,  align = 'left', xanchor = 'left',
                                       yref = 'paper', y = 1.20, valign = 'top'))

# Kyoto Protocol at 1997
temp_country_graph.add_shape(dict(type = 'line' , line = dict(color="black",width=0.25),
                                  xref = 'x'    , x0 = 1997 , x1=1997, 
                                  yref = 'paper', y0 = 0 , y1=1.2))
temp_country_graph.add_annotation(dict(text = '<b>Kyoto Protocol</b>', font=dict(size = 10), showarrow = False,
                                       xref = 'x'    , x = 1997,  align = 'left', xanchor = 'left',
                                       yref = 'paper', y = 1.20, valign = 'top'))

# add commitment period 
if temp_data['kyoto_annex_1'].unique()[0] == 'Annex I':
    temp_country_graph.add_shape(dict(type = 'rect' , fillcolor="lightgrey", opacity=0.5, layer="below", line_width=0,
                                  xref = 'x'    , x0 = 2008 , x1=2012.99, 
                                  yref = 'paper', y0 = 0 , y1=1.2))
    temp_country_graph.add_annotation(dict(text = '<b>Period 1</b>', font=dict(size = 10), showarrow = False,
                                           xref = 'x'    , x = 2010.5,  align = 'center',
                                           yref = 'paper', y = 1.2, valign = 'top'))


## set axis tatle
temp_country_graph.update_yaxes(title_text='<b>GtCO<sub>2</sub>-eq</b>')

## update figure layout
temp_country_graph.update_layout( title={'text': 'Non-Annex B - Emissions Trends',
                                         'x':0.5,
                                         'y':0.95,
                                         'font': dict(size=20),
                                         'xanchor': 'center',
                                         'yanchor': 'top'},
                                  yaxis_title="<b>GtCO<sub>2</sub>-eq</b>",template='plotly_white', 
                                  font=dict(family="Times",size=12, color='black'),
                                  margin=dict(l=75, r=25, t=100, b=50, pad=5),
                                  width=500, height=400,
                                  legend=dict(x=0.2, y=-0.1),
                                  legend_orientation="h")

## set range conditions
temp_country_graph.update_xaxes(range=[1989.75, 2015.25], showgrid=False)
    
temp_country_graph.update_yaxes(range=[0, 50.5], dtick = 10)

##  Show & Save graph
temp_country_graph.show()
temp_country_graph.write_image('/Users/david/OneDrive/Universität/Masterarbeit/Graphics/GHG_Non-Annex-B.png', scale=10)

# cleanup
del temp_country_graph
del temp_data

______________
# Gravity Model Estimation

In [ ]:
# Check dataset
data_gme.loc[data_gme['iso3_o'] == 'CHN'].loc[data_gme['iso3_d'] == 'CHE']

_____________
## Prepare Model Data

### USITC Dynamic Gravity Dataset

In [ ]:
gme_suppl_all = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1978_1992.csv')
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1993_2004.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_2005_2016.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.loc[gme_suppl_all['year'] >= 1990].loc[gme_suppl_all['year'] <= 2015]

In [ ]:
gme_suppl_all.iso3_o = gme_suppl_all.iso3_o.replace({"ZAR": "COD"})
gme_suppl_all.iso3_d = gme_suppl_all.iso3_d.replace({"ZAR": "COD"})
gme_suppl_all[['iso3_o', 'iso3_d', 'year']].sort_values(by=['iso3_o', 'iso3_d', 'year'])

### Country Data

In [ ]:
countries_gme = pd.read_excel('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/countries_gme.xlsx')
del countries_gme['country']
countries_gme = pd.merge(countries_gme, countries_gme, how='outer', on=['year'], sort=True,
         suffixes=('_o', '_d'), indicator=True, validate='many_to_many').rename(columns={'_merge': '_merge_2'})

### Merge

In [ ]:
gravity_data = pd.merge(data_gme, gme_suppl_all, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_1'})
gravity_data = pd.merge(countries_gme, gravity_data, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_3'})

gravity_data.drop(gravity_data[gravity_data['iso3_o'] == gravity_data['iso3_d']].index, inplace = True) 
gravity_data

del data_gme
del gme_suppl_all
del countries_gme

In [ ]:
gravity_data.describe(include = 'all').T

___________
## Prepare Variables

In [ ]:
gravity_data["log_distance"] = np.log(gravity_data["distance"])
gravity_data["log_pop_o"] = np.log(gravity_data["pop_o"]*1000000)
gravity_data["log_pop_d"] = np.log(gravity_data["pop_d"]*1000000)
gravity_data["log_gdp_wdi_cap_const_o"] = np.log(gravity_data["gdp_wdi_cap_const_o"])
gravity_data["log_gdp_wdi_cap_const_d"] = np.log(gravity_data["gdp_wdi_cap_const_d"])

gravity_data['kyoto_ratified_joint'] = (gravity_data['year_kyoto_ratified_o'] == 1) & (gravity_data['year_kyoto_ratified_d'] == 1)
gravity_data['kyoto_ratified_joint'] =  gravity_data['kyoto_ratified_joint'].astype(int)

In [ ]:
gravity_data = gravity_data[['iso3_d',
                             'iso3_o',
                             'ghg_co2eq',
                             'ghg_co2eq_fpa',
                             'year',
                             'log_gdp_wdi_cap_const_o',
                             'log_gdp_wdi_cap_const_d',
                             'log_pop_o',
                             'log_pop_d',
                             'log_distance',
                             'contiguity',
                             'common_language', 
                             'member_wto_joint',
                             'member_eu_o',
                             'member_eu_d',
                             'member_eu_joint',
                             'agree_pta',
                             'year_kyoto_ratified_o',
                             'year_kyoto_ratified_d',
                             'kyoto_ratified_joint',
                             'kyoto_annex_b_o',
                             'kyoto_annex_b_d']]
gravity_data = gravity_data[gravity_data.ghg_co2eq.notnull() | gravity_data.ghg_co2eq_fpa.notnull()]

In [ ]:
gravity_data.loc[gravity_data['iso3_o'] == 'CHN'].loc[gravity_data['iso3_d'] == 'CHE']

_______________
## Initialize GME

In [ ]:
gme_data    = gme.EstimationData(data_frame=gravity_data,
                              imp_var_name='iso3_d',
                              exp_var_name='iso3_o',
                              trade_var_name='ghg_co2eq',
                              year_var_name='year')

gme_data_ay = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_d'] == 1],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

gme_data_an = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_d'] == 0],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

################################################################################################################

gme_data_ya = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 1],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

gme_data_yy = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 1]
                                                        .loc[gravity_data['kyoto_annex_b_d'] == 1],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

gme_data_yn = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 1]
                                                        .loc[gravity_data['kyoto_annex_b_d'] == 0],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

################################################################################################################

gme_data_na = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 0],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

gme_data_ny = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 0]
                                                        .loc[gravity_data['kyoto_annex_b_d'] == 1],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')

gme_data_nn = gme.EstimationData(data_frame=gravity_data.loc[gravity_data['kyoto_annex_b_o'] == 0]
                                                        .loc[gravity_data['kyoto_annex_b_d'] == 0],
                                 imp_var_name='iso3_d',
                                 exp_var_name='iso3_o',
                                 trade_var_name='ghg_co2eq',
                                 year_var_name='year')


In [ ]:
gme_data

In [ ]:
gme_data.info()

________
## Baseline Models

In [ ]:
model_baseline = gme.EstimationModel(estimation_data = gme_data,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline = model_baseline.estimate()
estimates_baseline['all'].summary()

In [ ]:
model_baseline_ay = gme.EstimationModel(estimation_data = gme_data_ay,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_ay = model_baseline_ay.estimate()
estimates_baseline_ay['all'].summary()

In [ ]:
model_baseline_an = gme.EstimationModel(estimation_data = gme_data_an,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_an = model_baseline_an.estimate()
estimates_baseline_an['all'].summary()

In [ ]:
model_baseline_ya = gme.EstimationModel(estimation_data = gme_data_ya,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_ya = model_baseline_ya.estimate()
estimates_baseline_ya['all'].summary()

In [ ]:
model_baseline_yy = gme.EstimationModel(estimation_data = gme_data_yy,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_yy = model_baseline_yy.estimate()
estimates_baseline_yy['all'].summary()

In [ ]:
model_baseline_yn = gme.EstimationModel(estimation_data = gme_data_yn,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_yn = model_baseline_yn.estimate()
estimates_baseline_yn['all'].summary()

In [ ]:
model_baseline_na = gme.EstimationModel(estimation_data = gme_data_na,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_na = model_baseline_na.estimate()
estimates_baseline_na['all'].summary()

In [ ]:
model_baseline_ny = gme.EstimationModel(estimation_data = gme_data_ny,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_ny = model_baseline_ny.estimate()
estimates_baseline_ny['all'].summary()

In [ ]:
model_baseline_nn = gme.EstimationModel(estimation_data = gme_data_nn,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_nn = model_baseline_nn.estimate()
estimates_baseline_nn['all'].summary()

### diagnostics

In [ ]:
model_baseline.ppml_diagnostics

In [ ]:
model_baseline_ay.ppml_diagnostics

In [ ]:
model_baseline_an.ppml_diagnostics

In [ ]:
model_baseline_ya.ppml_diagnostics

In [ ]:
model_baseline_yy.ppml_diagnostics

In [ ]:
model_baseline_yn.ppml_diagnostics

In [ ]:
model_baseline_na.ppml_diagnostics

In [ ]:
model_baseline_ny.ppml_diagnostics

In [ ]:
model_baseline_nn.ppml_diagnostics

__________
## Fixed Effects Model

In [ ]:
model_fe = gme.EstimationModel(estimation_data = gme_data,
                                       lhs_var = 'ghg_co2eq',
                                       rhs_var = ['log_distance',
                                                  'contiguity',
                                                  'common_language', 
                                                  'member_wto_joint',
                                                  'agree_pta',
                                                  'kyoto_ratified_joint'],
                                 fixed_effects = [['iso3_d','year'],
                                                  ['iso3_o','year']])

In [ ]:
# THIS REQUIRES A LOT OF MEMORY - RUN WITH CAUTION
#estimates_fe = model_fe.estimate()

In [ ]:
#estimates_fe['all'].summary()

In [ ]:
#model_fe.ppml_diagnostics

___________________
## FPA Specification

In [ ]:
gme_data_fpa = gme.EstimationData(data_frame=gravity_data,
                                  imp_var_name='iso3_d',
                                  exp_var_name='iso3_o',
                                  trade_var_name='ghg_co2eq_fpa',
                                  year_var_name='year')

In [ ]:
gme_data_fpa

In [ ]:
gme_data_fpa.info()

In [ ]:
model_baseline_fpa = gme.EstimationModel(estimation_data = gme_data_fpa,
                                             lhs_var = 'ghg_co2eq_fpa',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_fpa = model_baseline_fpa.estimate()
estimates_baseline_fpa['all'].summary()

In [ ]:
model_baseline_fpa.ppml_diagnostics

_____________________
## Importer or Exporter Ratified


In [ ]:
model_baseline_alt_kyoto = gme.EstimationModel(estimation_data = gme_data,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'year_kyoto_ratified_o',
                                                        'year_kyoto_ratified_d'])
estimates_baseline_alt_kyoto = model_baseline_alt_kyoto.estimate()
estimates_baseline_alt_kyoto['all'].summary()

In [ ]:
model_baseline_alt_kyoto.ppml_diagnostics

_____________________
## EU Specification

In [ ]:
model_baseline_eu2 = gme.EstimationModel(estimation_data = gme_data,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'year_kyoto_ratified_o',
                                                        'year_kyoto_ratified_d',
                                                        'member_eu_o',
                                                        'member_eu_d'])
estimates_baseline_eu2 = model_baseline_eu2.estimate()
estimates_baseline_eu2['all'].summary()

In [ ]:
model_baseline_eu2.ppml_diagnostics

______________
## Exiobase Data

In [ ]:
convert_ISO_3166_2_to_1 = {
'AF':'AFG','AX':'ALA','AL':'ALB','DZ':'DZA','AS':'ASM','AD':'AND','AO':'AGO','AI':'AIA',
'AQ':'ATA','AG':'ATG','AR':'ARG','AM':'ARM','AW':'ABW','AU':'AUS','AT':'AUT','AZ':'AZE',
'BS':'BHS','BH':'BHR','BD':'BGD','BB':'BRB','BY':'BLR','BE':'BEL','BZ':'BLZ','BJ':'BEN',
'BM':'BMU','BT':'BTN','BO':'BOL','BA':'BIH','BW':'BWA','BV':'BVT','BR':'BRA','IO':'IOT',
'BN':'BRN','BG':'BGR','BF':'BFA','BI':'BDI','KH':'KHM','CM':'CMR','CA':'CAN','CV':'CPV',
'KY':'CYM','CF':'CAF','TD':'TCD','CL':'CHL','CN':'CHN','CX':'CXR','CC':'CCK','CO':'COL',
'KM':'COM','CG':'COG','CD':'COD','CK':'COK','CR':'CRI','CI':'CIV','HR':'HRV','CU':'CUB',
'CY':'CYP','CZ':'CZE','DK':'DNK','DJ':'DJI','DM':'DMA','DO':'DOM','EC':'ECU','EG':'EGY',
'SV':'SLV','GQ':'GNQ','ER':'ERI','EE':'EST','ET':'ETH','FK':'FLK','FO':'FRO','FJ':'FJI',
'FI':'FIN','FR':'FRA','GF':'GUF','PF':'PYF','TF':'ATF','GA':'GAB','GM':'GMB','GE':'GEO',
'DE':'DEU','GH':'GHA','GI':'GIB','GR':'GRC','GL':'GRL','GD':'GRD','GP':'GLP','GU':'GUM',
'GT':'GTM','GG':'GGY','GN':'GIN','GW':'GNB','GY':'GUY','HT':'HTI','HM':'HMD','VA':'VAT',
'HN':'HND','HK':'HKG','HU':'HUN','IS':'ISL','IN':'IND','ID':'IDN','IR':'IRN','IQ':'IRQ',
'IE':'IRL','IM':'IMN','IL':'ISR','IT':'ITA','JM':'JAM','JP':'JPN','JE':'JEY','JO':'JOR',
'KZ':'KAZ','KE':'KEN','KI':'KIR','KP':'PRK','KR':'KOR','KW':'KWT','KG':'KGZ','LA':'LAO',
'LV':'LVA','LB':'LBN','LS':'LSO','LR':'LBR','LY':'LBY','LI':'LIE','LT':'LTU','LU':'LUX',
'MO':'MAC','MK':'MKD','MG':'MDG','MW':'MWI','MY':'MYS','MV':'MDV','ML':'MLI','MT':'MLT',
'MH':'MHL','MQ':'MTQ','MR':'MRT','MU':'MUS','YT':'MYT','MX':'MEX','FM':'FSM','MD':'MDA',
'MC':'MCO','MN':'MNG','ME':'MNE','MS':'MSR','MA':'MAR','MZ':'MOZ','MM':'MMR','NA':'NAM',
'NR':'NRU','NP':'NPL','NL':'NLD','AN':'ANT','NC':'NCL','NZ':'NZL','NI':'NIC','NE':'NER',
'NG':'NGA','NU':'NIU','NF':'NFK','MP':'MNP','NO':'NOR','OM':'OMN','PK':'PAK','PW':'PLW',
'PS':'PSE','PA':'PAN','PG':'PNG','PY':'PRY','PE':'PER','PH':'PHL','PN':'PCN','PL':'POL',
'PT':'PRT','PR':'PRI','QA':'QAT','RE':'REU','RO':'ROU','RU':'RUS','RW':'RWA','BL':'BLM',
'SH':'SHN','KN':'KNA','LC':'LCA','MF':'MAF','PM':'SPM','VC':'VCT','WS':'WSM','SM':'SMR',
'ST':'STP','SA':'SAU','SN':'SEN','RS':'SRB','SC':'SYC','SL':'SLE','SG':'SGP','SK':'SVK',
'SI':'SVN','SB':'SLB','SO':'SOM','ZA':'ZAF','GS':'SGS','ES':'ESP','LK':'LKA','SD':'SDN',
'SR':'SUR','SJ':'SJM','SZ':'SWZ','SE':'SWE','CH':'CHE','SY':'SYR','TW':'TWN','TJ':'TJK',
'TZ':'TZA','TH':'THA','TL':'TLS','TG':'TGO','TK':'TKL','TO':'TON','TT':'TTO','TN':'TUN',
'TR':'TUR','TM':'TKM','TC':'TCA','TV':'TUV','UG':'UGA','UA':'UKR','AE':'ARE','GB':'GBR',
'US':'USA','UM':'UMI','UY':'URY','UZ':'UZB','VU':'VUT','VE':'VEN','VN':'VNM','VG':'VGB',
'VI':'VIR','WF':'WLF','EH':'ESH','YE':'YEM','ZM':'ZMB','ZW':'ZWE'
}

### MRIO System

In [ ]:
print('Preparing import...', end="")
time_start_all = time.time()

c =   ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 
       'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 
       'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'GB', 'US', 'JP', 
       'CN', 'CA', 'KR', 'BR', 'IN', 'MX', 'RU', 'AU', 'CH', 'TR', 
       'TW', 'NO', 'ID', 'ZA']

data_gme = pd.DataFrame(columns=['iso3_o', 'iso3_d', 'co2_kg', 'ch4_kg', 'year'])
graph_countries_data = pd.read_excel('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/incl_countries.xlsx')
graph_countries = graph_countries_data['country']
graph_ghg_cba = pd.DataFrame(graph_countries)

print(' starting...')

for year_i in range(1995, 2012):
    
    time_start_loop = time.time()
    
    ## Initialize MRIO
    print('Year ' + str(year_i) + ': Loading... mrio', end="")
    temp_mrio = pymrio.parse_exiobase3(path='/Users/david/OneDrive/Universität/Masterarbeit/Data/mrio/exio3/exiobase3.4_iot_'+ str(year_i) +'_pxp.zip')
    print(' - CO2', end="")
    temp_mrio.co2_diag = temp_mrio.satellite.diag_stressor('CO2 - combustion - air')
    print(' - CH4', end="")
    temp_mrio.ch4_diag = temp_mrio.satellite.diag_stressor('CH4 - combustion - air')
    print(' - calc', end="")
    temp_mrio.calc_all()
    
    ## Calculate Emissions Flows and Accounts
    print(' / flows', end="")
    temp_cba_co2 = pd.DataFrame(temp_mrio.co2_diag.M.sort_index().dot(temp_mrio.Y.groupby(level='region', axis=1).sum().sort_index()))
    temp_cba_ch4 = pd.DataFrame(temp_mrio.ch4_diag.M.sort_index().dot(temp_mrio.Y.groupby(level='region', axis=1).sum().sort_index()))

    ## GME Dataframe
    print(' / gme - CO2', end="")
    temp_gme_co2 = pd.DataFrame(temp_cba_co2.groupby(level='region', axis=0).sum().stack(0), columns = ['co2_kg'])
    temp_gme_co2 = temp_gme_co2.reset_index(level=1).rename(columns={'region': 'iso3_d'})
    temp_gme_co2 = temp_gme_co2.reset_index(level=0).rename(columns={'region': 'iso3_o'})
    temp_gme_co2 = temp_gme_co2[temp_gme_co2['iso3_o'].isin(c)]    
    temp_gme_co2 = temp_gme_co2[temp_gme_co2['iso3_d'].isin(c)]
    temp_gme_co2['iso3_o'] = temp_gme_co2.iso3_o.map(convert_ISO_3166_2_to_1)
    temp_gme_co2['iso3_d'] = temp_gme_co2.iso3_d.map(convert_ISO_3166_2_to_1)
    temp_gme_co2['year'] = year_i
    print(' - CH4', end="")
    temp_gme_ch4 = pd.DataFrame(temp_cba_ch4.groupby(level='region', axis=0).sum().stack(0), columns = ['ch4_kg'])
    temp_gme_ch4 = temp_gme_ch4.reset_index(level=1).rename(columns={'region': 'iso3_d'})
    temp_gme_ch4 = temp_gme_ch4.reset_index(level=0).rename(columns={'region': 'iso3_o'})
    temp_gme_ch4 = temp_gme_ch4[temp_gme_ch4['iso3_o'].isin(c)]    
    temp_gme_ch4 = temp_gme_ch4[temp_gme_ch4['iso3_d'].isin(c)]
    temp_gme_ch4['iso3_o'] = temp_gme_ch4.iso3_o.map(convert_ISO_3166_2_to_1)
    temp_gme_ch4['iso3_d'] = temp_gme_ch4.iso3_d.map(convert_ISO_3166_2_to_1)
    temp_gme_ch4['year'] = year_i
    temp_gme = pd.merge(temp_gme_co2, temp_gme_ch4, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    data_gme = data_gme.append(temp_gme[['iso3_o', 'iso3_d', 'co2_kg', 'ch4_kg', 'year']], ignore_index=True)

    ## Country CBA
    print(' / cba', end="")
    temp_cba_co2 = temp_cba_co2.sum(axis=0).reset_index(level=0).rename(columns={'region': 'iso3', 0: 'CBA kgCO2eq'})
    temp_cba_co2 = temp_cba_co2[temp_cba_co2['iso3'].isin(c)]
    temp_cba_co2['iso3'] = temp_cba_co2.iso3.map(convert_ISO_3166_2_to_1)
    temp_cba_co2['year'] = year_i
    temp_cba_co2 = pd.merge(graph_countries_data, temp_cba_co2, how='left', on=['iso3'], sort=True, suffixes=('_x', '_y'), indicator=False, validate='one_to_one')
    graph_ghg_cba = pd.concat([graph_ghg_cba, temp_cba_co2], ignore_index=True, axis=0, join='outer')
    graph_ghg_cba = graph_ghg_cba[graph_ghg_cba['year'].notna()]
    
    ## Clean Memory
    print(' ... cleanup', end="")
    del temp_mrio
    del temp_gme
    del temp_gme_co2
    del temp_gme_ch4
    del temp_cba_co2
    print(' ... done after: ', str(datetime.timedelta(seconds=round(time.time() - time_start_loop))))

print('Finishing touches ...', end="")
del graph_countries_data
del graph_countries

print(' done. Total time:', str(datetime.timedelta(seconds=round(time.time() - time_start_all))))

In [ ]:
graph_ghg_cba.loc[graph_ghg_cba['iso3'] == 'CHE']

In [ ]:
data_gme['ghg_co2eq'] = ((data_gme['co2_kg'] + (data_gme['ch4_kg']*25 )) / 1000000)
data_gme.loc[data_gme['iso3_o'] == 'CHN'].loc[data_gme['iso3_d'] == 'CHE']

### USITC Dynamic Gravity Dataset

In [ ]:
gme_suppl_all = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1978_1992.csv')
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1993_2004.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_2005_2016.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.loc[gme_suppl_all['year'] >= 1995].loc[gme_suppl_all['year'] <= 2011]

In [ ]:
gme_suppl_all.iso3_o = gme_suppl_all.iso3_o.replace({"ZAR": "COD"})
gme_suppl_all.iso3_d = gme_suppl_all.iso3_d.replace({"ZAR": "COD"})
gme_suppl_all[['iso3_o', 'iso3_d', 'year']].sort_values(by=['iso3_o', 'iso3_d', 'year'])

### Country Data

In [ ]:
countries_gme = pd.read_excel('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/countries_gme.xlsx')
del countries_gme['country']
countries_gme = pd.merge(countries_gme, countries_gme, how='outer', on=['year'], sort=True,
         suffixes=('_o', '_d'), indicator=True, validate='many_to_many').rename(columns={'_merge': '_merge_2'})

### Merge

In [ ]:
gravity_data = pd.merge(data_gme, gme_suppl_all, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_1'})
gravity_data = pd.merge(countries_gme, gravity_data, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_3'})

gravity_data.drop(gravity_data[gravity_data['iso3_o'] == gravity_data['iso3_d']].index, inplace = True) 
gravity_data

#del data_gme
del gme_suppl_all
del countries_gme

In [ ]:
gravity_data.describe(include = 'all').T

### Prepare Variables

In [ ]:
gravity_data["log_distance"] = np.log(gravity_data["distance"])
gravity_data["log_pop_o"] = np.log(gravity_data["pop_o"]*1000000)
gravity_data["log_pop_d"] = np.log(gravity_data["pop_d"]*1000000)
gravity_data["log_gdp_wdi_cap_const_o"] = np.log(gravity_data["gdp_wdi_cap_const_o"])
gravity_data["log_gdp_wdi_cap_const_d"] = np.log(gravity_data["gdp_wdi_cap_const_d"])

gravity_data['kyoto_ratified_joint'] = (gravity_data['year_kyoto_ratified_o'] == 1) & (gravity_data['year_kyoto_ratified_d'] == 1)
gravity_data['kyoto_ratified_joint'] =  gravity_data['kyoto_ratified_joint'].astype(int)

In [ ]:
gravity_data = gravity_data[['iso3_d',
                             'iso3_o',
                             'ghg_co2eq',
                             'year',
                             'log_gdp_wdi_cap_const_o',
                             'log_gdp_wdi_cap_const_d',
                             'log_pop_o',
                             'log_pop_d',
                             'log_distance',
                             'contiguity',
                             'common_language', 
                             'member_wto_joint',
                             'member_eu_joint',
                             'agree_pta',
                             'kyoto_ratified_joint',
                             'kyoto_annex_b_o',
                             'kyoto_annex_b_d']]

In [ ]:
gravity_data

### GME Model

In [ ]:
gme_data_exio = gme.EstimationData(data_frame=gravity_data[gravity_data.ghg_co2eq.notnull()],
                                  imp_var_name='iso3_d',
                                  exp_var_name='iso3_o',
                                  trade_var_name='ghg_co2eq',
                                  year_var_name='year')

In [ ]:
gme_data_exio

In [ ]:
gme_data_exio.info()

In [ ]:
model_baseline_exio = gme.EstimationModel(estimation_data = gme_data_exio,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_exio = model_baseline_exio.estimate()
estimates_baseline_exio['all'].summary()

In [ ]:
model_baseline_exio.ppml_diagnostics

______________
## GTAP Data

In [ ]:
gtap_co2 = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/mrio/gtap/co2_trade_c_oct19.csv')
gtap_ch4 = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/mrio/gtap/ch4_trade_c_oct19.csv')
data_gme = pd.merge(gtap_co2, 
                     gtap_ch4, 
                     how='left', 
                     on=['reg_i', 'reg_j', 'yr'], 
                     validate='one_to_one', 
                     suffixes=('_x', '_y'), 
                     sort=True,
                     indicator=False)
drop_reg = ['xac','xap','xcm','xef','xme','xnf','xsa','xsc','xsm','xss','xsu','xtw']
data_gme = data_gme[(~data_gme['reg_i'].isin(drop_reg))]
data_gme = data_gme[(~data_gme['reg_j'].isin(drop_reg))]
data_gme = data_gme[data_gme['reg_i'] != data_gme['reg_j']] 
data_gme['reg_i'] = data_gme['reg_i'].str.upper()
data_gme['reg_j'] = data_gme['reg_j'].str.upper()
data_gme = data_gme.rename(columns={'reg_i': 'iso3_o','reg_j': 'iso3_d','yr': 'year'})
data_gme['ghg_co2eq'] = ((data_gme['co2_ij'] + data_gme['ch4_ij']))
data_gme

In [ ]:
# Check dataset
data_gme.loc[data_gme['iso3_o'] == 'CHN'].loc[data_gme['iso3_d'] == 'CHE']

In [ ]:
gme_suppl_all = pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1978_1992.csv')
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_1993_2004.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.append(pd.read_csv('/Users/david/OneDrive/Universität/Masterarbeit/Data/gme/usitc dynamic gravity dataset/release_1.0_2005_2016.csv'), ignore_index=True)
gme_suppl_all = gme_suppl_all.loc[gme_suppl_all['year'] >= 1997].loc[gme_suppl_all['year'] <= 2014]
gme_suppl_all.iso3_o = gme_suppl_all.iso3_o.replace({"ZAR": "COD"})
gme_suppl_all.iso3_d = gme_suppl_all.iso3_d.replace({"ZAR": "COD"})
gme_suppl_all[['iso3_o', 'iso3_d', 'year']].sort_values(by=['iso3_o', 'iso3_d', 'year'])
countries_gme = pd.read_excel('/Users/david/OneDrive/Universität/Masterarbeit/Data/countries/countries_gme.xlsx')
del countries_gme['country']
countries_gme = pd.merge(countries_gme, countries_gme, how='outer', on=['year'], sort=True,
         suffixes=('_o', '_d'), indicator=True, validate='many_to_many').rename(columns={'_merge': '_merge_2'})
gravity_data = pd.merge(data_gme, gme_suppl_all, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_1'})
gravity_data = pd.merge(countries_gme, gravity_data, how='left', on=['iso3_o', 'iso3_d', 'year'], sort=True,
         suffixes=('_x', '_y'), indicator=True, validate='one_to_one').rename(columns={'_merge': '_merge_3'})

gravity_data.drop(gravity_data[gravity_data['iso3_o'] == gravity_data['iso3_d']].index, inplace = True) 
gravity_data

del data_gme
del gme_suppl_all
del countries_gme

In [ ]:
gravity_data.describe(include = 'all').T

In [ ]:
gravity_data["log_distance"] = np.log(gravity_data["distance"])
gravity_data["log_pop_o"] = np.log(gravity_data["pop_o"]*1000000)
gravity_data["log_pop_d"] = np.log(gravity_data["pop_d"]*1000000)
gravity_data["log_gdp_wdi_cap_const_o"] = np.log(gravity_data["gdp_wdi_cap_const_o"])
gravity_data["log_gdp_wdi_cap_const_d"] = np.log(gravity_data["gdp_wdi_cap_const_d"])

gravity_data['kyoto_ratified_joint'] = (gravity_data['year_kyoto_ratified_o'] == 1) & (gravity_data['year_kyoto_ratified_d'] == 1)
gravity_data['kyoto_ratified_joint'] =  gravity_data['kyoto_ratified_joint'].astype(int)

In [ ]:
gravity_data = gravity_data[['iso3_d',
                             'iso3_o',
                             'ghg_co2eq',
                             'year',
                             'log_gdp_wdi_cap_const_o',
                             'log_gdp_wdi_cap_const_d',
                             'log_pop_o',
                             'log_pop_d',
                             'log_distance',
                             'contiguity',
                             'common_language', 
                             'member_wto_joint',
                             'member_eu_joint',
                             'agree_pta',
                             'kyoto_ratified_joint',
                             'kyoto_annex_b_o',
                             'kyoto_annex_b_d']]

In [ ]:
gravity_data

In [ ]:
gme_data_gtap = gme.EstimationData(data_frame=gravity_data[gravity_data.ghg_co2eq.notnull()],
                                  imp_var_name='iso3_d',
                                  exp_var_name='iso3_o',
                                  trade_var_name='ghg_co2eq',
                                  year_var_name='year')

In [ ]:
gme_data_gtap

In [ ]:
gme_data_gtap.info()

In [ ]:
model_baseline_gtap = gme.EstimationModel(estimation_data = gme_data_gtap,
                                             lhs_var = 'ghg_co2eq',
                                             rhs_var = ['log_gdp_wdi_cap_const_o',
                                                        'log_gdp_wdi_cap_const_d',
                                                        'log_pop_o',
                                                        'log_pop_d',
                                                        'log_distance',
                                                        'contiguity',
                                                        'common_language', 
                                                        'member_wto_joint',
                                                        'agree_pta',
                                                        'kyoto_ratified_joint'])
estimates_baseline_gtap = model_baseline_gtap.estimate()
estimates_baseline_gtap['all'].summary()

In [ ]:
model_baseline_gtap.ppml_diagnostics